### installation 

In [1]:
!pip install pythainlp

In [2]:
!pip install epitran

In [141]:
#!pip3 install numpy --upgrade

In [142]:
# !pip install python-crfsuite

In [72]:
!pip install attacut

     ---------------------------------------- 1.3/1.3 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 473.8/473.8 kB 9.9 MB/s eta 0:00:00
     ---------------------------------------- 88.3/88.3 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=63178df46528c61793a0f8605bb27537ca3bf2cfdf3b95f6e4ddc58bd8f64a18
  Stored in directory: c:\users\kakas\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116933 sha256=40d7c8d02b4b30fc7bd1824bac47e29ce8f6fdf494ffe481396f81be42bf9276
  Stored in directory: c:\users\kakas\appdata\local\pip\cache\wheels\f7\f1\89\b9ea2bf8f80ec027a88fef1d354b3816b4d3d29530988972f6
Su

In [3]:
!pip install deepcut

In [4]:
from pythainlp import correct , spell

### imports 

In [48]:
import string
import numpy as np
import re
import os 
import pythainlp
import glob
import tqdm

In [6]:
from pythainlp import word_tokenize
from pythainlp.util import normalize
from pythainlp import sent_tokenize
from pythainlp.tag import pos_tag, pos_tag_sents
#from pythainlp.tag.named_entity import ThaiNameTagger

In [7]:
import deepcut

In [73]:
from attacut import tokenize

In [ ]:
from pythainlp.spell import correct
from pythainlp.spell import correct_sent

In [8]:
pythainlp.__version__

'3.1.1'

### text cleaning 

In [9]:
class TextCleaner():
    """
    Thai text cleaner
    """
    def __init__(self):        
        self.min_ascii = '0E01'
        #self.max_ascii = '0E7F'
        
        self.max_ascii = '0E5B'
        
        self.punctuation = string.punctuation + '،' + '؛' + '؟' + '؛' + '۔' + '»' + '«' + '-'
        self.one_space_regex = r"\s((\s)(\s+)?)?"
        self.multiple_space_regex = r"\s+"
        self.text = None

        self.dict_punct = dict(zip(list(self.punctuation), np.repeat(' ', len(self.punctuation))))
        
    def remove_punct(self, text):
        table = str.maketrans(self.dict_punct)
        text = text.translate(table)
        return text
    
    def remove_num(self, text):
        num_pattern = r'[0-9]'
        #num_pattern = r'[\u06F0-\u06F9]'
        text = re.sub(num_pattern, '', text)
        return text
    
    def remove_spaces(self, text):
        try:
            text = re.sub(r'(\n){2,}','#', text)
            full_text = ""
            sentences = text.split("#")
            for sen in sentences:
                sen = re.sub(self.multiple_space_regex, '', sen)
                full_text = full_text + sen + '#'
            #text = os.linesep.join([s + '\n' for s in String.split("\\n\\n") if s])
            #remove all whitespaces since spaces are considered as a sentence seperator in thai
#             sentences = text.split("\\n\\n")
#             full_text =""
#             for sen in sentences:
#                 sen = re.sub(self.multiple_space_regex, '', sen)
#                 full_text = full_text + sen + ' '
            #text = re.sub(self.multiple_space_regex, '', text)
            #return text
            return full_text[:-1]
        
        except IndexError as e:
            return ''
    
#     def is_fa_token(self, token):
#         for ch in set(token):
#             if ord(ch) < int(self.min_ascii, 16) or ord(ch) > int(self.max_ascii, 16):
#                 return False
#         return True

    def remove_foreign_lang(self, text):
        clean_text = ''
        for token in text.split('#'):
            for ch in token:
                if ord(ch) > int(self.min_ascii, 16) and ord(ch) < int(self.max_ascii, 16):
                    clean_text += ch
            clean_text+= ' '
                
        return clean_text[:-1]
    
    def normalize_func (self , text):
        cleaned_normalized = normalize(text)
        return cleaned_normalized
    
    def clean_text(self, text):
        #print ("raw")
        #print (len(text))
        #print (text)
        text = self.remove_punct(text)
        #print("no punc")
        #print (len(text))
        #print (text)
        text = self.remove_num(text)
        #print ("no num")
        #print (len(text))
        #print (text)
        text = self.remove_spaces(text)
        #print ("no spaces")
        #print (len(text))
        #print (text)
        text = self.remove_foreign_lang(text)
        #print ("no forign")
        #print (len(text))
        #print (text)
        text = self.normalize_func(text)
        #print ("normalized")
        #print (len(text))
        #print (text)
        return text

### example 

In [10]:
# Open a file: file
file = open('example.txt',mode='r',encoding = 'utf-8')
 
# read all lines at once
all_of_it = file.read()
 
# close the file
file.close()

In [11]:
all_of_it

"ข ๕ = ๓ % ๓ ฆ ร 3 = ๑ 7 ส ข =% = ฆ = ๐ = \nร า ย ได ้ ท ี ม ั ง ค ั ง แล ะ ม ั น ค ง เป ็ น บ ุ ค ล า ก ร ท ี ม ี ว ิ น ั ย เป ี ย ม ไป ด ้ ว ย ค ุ ณ ธ ร ร ม แล ะ จ ร ิ ย ธร ร ม แล ะ ม ี จ ิ ต ส ํ า น ึ ก \nร ั บ ผิ ด ชอบ ต ่ อ ต น เอ ง แล ะ ผู ้ อ ื ่ น ส ํ า น ั ก ง า น ก ศ น . จ ึ ง ได ้ พ ิ จ า ร ณา ท บ ท ว น ห ล ั ก ก า ร จ ุ ด ห ม า ย ม า ต ร ฐา น ผล ก า ร \n' 4 ข ” \nเร ี ย น ร ู ้ ท ี ค า ด ห ว ั ง แล ะ เน ื ้ อ ห า ส า ร ะ ท ั ง 5 ก ล ุ ่ ม ส า ร ะ ก า ร เร ี ย น ร ู ้ ขอ ง ห ล ั ก ส ู ต ร ก า ร ศึ ก ษา น อ ก ร ะ บ บ ร ะ ด ั บ \nข ” 4 \n\nก า ร ศึ ก ษา ขั น พ ื ่ น ฐา น พ ุ ท ธ ศ ั ก ร า ช 2551 ให ้ ม ี ค ว า ม ส อ ด ค ล ้ อ ง ต อ บ ส น อ ง น โย บ า ย ก ร ะ ท ร ว ง ศึ ก ษา ธิ ก า ร \n<= ป ข % ๆ ๆ ขม ข ๓ ข ๆ « = = ม ซ่ ขม = \nซ ชง ส ่ ง ผล ให ้ ต ้ อ ง ป ร ั บ ป ร ุ ง ห น ั ง ส ื อ เร ี ย น โด ย ก า ร เพ ิ ่ ม แล ะ ส อ ด แท ร ก เน ื ้ อ ห า ส า ร ะ เก ี ย ว ก ั บ อ า ชี พ ค ุ ณ ธ ร ร ม \n=% = บุ 1 = =% = ี จ -. ๑ ๓ (๓ ' \nจ ร ิ ย ธร ร ม แล ะ ก า ร เต ร ี ย ม พ ร ้ อ ม เพ ื ่ อ

In [12]:
len(all_of_it)

1662

In [13]:
cleaner = TextCleaner()

In [14]:
cleaned = cleaner.clean_text(all_of_it)

In [15]:
len(cleaned)

767

In [16]:
cleaned

'ข๕๓๓ฆร๑สขฆ๐รายได้ทีมังคังและมันคงเป็นบุคลารทีมีวินัยเปียมไปด้วยคุณธรรมและจริยธรรมและมีจิตสำนึรับผิดชอบต่อตนเองและผู้อื่นสำนังานศนจึงได้พิจารณาทบทวนหลัารจุดหมายมาตรฐานผลารขเรียนรู้ทีคาดหวังและเนื้อหาสาระทังลุ่มสาระารเรียนรู้ของหลัสูตรารศึษานอระบบระดับข ารศึษาขันพื่นฐานพุทธศัราชให้มีความสอดคล้องตอบสนองนโยบายระทรวงศึษาธิารปขๆๆขมข๓ขๆมซ่ขมซชงส่งผลให้ต้องปรับปรุงหนังสือเรียนโดยารเพิ่มและสอดแทรเนื้อหาสาระเียวับอาชีพคุณธรรมบุีจ๑๓๓จริยธรรมและารเตรียมพร้อมเพื่อเข้าสู่ประชาคมอาเซียนในรายวิชาทีมีความเียวข้องสัมพันธั์นแต่๑ข๕๑ขสงจขขขขขขณ๑๕ยังคงหลัารและวิธีารเดิมในารพัฒนาหนังสือทีให้ผู้เรียนศึษาค้นควำความรู้ด้วยตนเองปฏิบัติ๐ข๓ฆ้ขขขง๓สิจรรมทำแบบฝึหัดเพือทดสอบความรู้ความเข้าใจมีารอภิปรายแลเปลียนเรียนรูั้บลุ่มหรือี๐แขฆ้ฆ้ศึษาเพิ่มเต้มจาภูมิปัญญาท้องถันแหล่งารเรียนรู้และสืออน'

In [17]:
# #no need for sentences tokenizer since we removed all the whitespaces


# print("default (crfcut):")
# print(sent_tokenize(cleaned))
# cleaned_senetences = sent_tokenize(cleaned)
# print (len(cleaned_senetences))

In [18]:
# cleaned_text = ""
# for sen in cleaned_senetences:
#     words = word_tokenize(sen , keep_whitespace=False)
#     for word in words:
#         cleaned_text += word +" "

In [19]:
# cleaned_text

In [32]:
cleaned_text = ""
words = word_tokenize(cleaned , keep_whitespace=False)
for word in words:
    cleaned_text += word +" "

In [33]:
cleaned_text

'ข ๕๓๓ ฆร ๑ สขฆ ๐ ราย ได้ที มัง คัง และ มัน คง เป็น บุ คลา ร ที มีวินัย เปีย ม ไป ด้วย คุณธรรม และ จริยธรรม และ มี จิต สำ นึ รับผิดชอบ ต่อ ตนเอง และ ผู้อื่น สำ นัง านศน จึง ได้ พิจารณา ทบทวน หลัาร จุดหมาย มาตรฐาน ผลา รข เรียน รู้ที คาดหวัง และ เนื้อหาสาระ ทัง ลุ่ม สาระ าร เรียนรู้ ของ หลัสู ตรา รศึษานอ ระบบ ระดับ ข ารศึษา ขัน พื่น ฐาน พุทธ ศัราช ให้ มี ความสอดคล้อง ตอบ สนองนโยบาย ระ ทรวง ศึษาธิารปขๆๆขมข ๓ ขๆมซ่ขมซชง ส่งผล ให้ ต้อง ปรับปรุง หนังสือเรียน โด ยา ร เพิ่ม และ สอด แทร เนื้อหาสาระ เีย วับ อาชีพ คุณธรรม บุ ีจ ๑๓๓ จริยธรรม และ าร เตรียมพร้อม เพื่อ เข้าสู่ ประชาคม อาเซียน ใน รายวิชา ที มี ความ เียว ข้อง สัม พัน ธั์น แต่ ๑ ข ๕๑ ขสงจขขขขขขณ ๑๕ ยังคง หลัาร และ วิธี าร เดิม ใน าร พัฒนา หนังสือ ที ให้ ผู้เรียน ศึษา ค้น ควำ ความรู้ ด้วย ตนเอง ปฏิบัติ ๐ ข ๓ ฆ้ขขขง ๓ สิ จรรม ทำ แบบ ฝึ หัด เพือ ทดสอบ ความรู้ ความเข้าใจ มี าร อภิปราย แล เปลียน เรียน รู ั้บ ลุ่ม หรือ ี ๐ แข ฆ้ฆ้ศึษา เพิ่ม เต้ม จา ภูมิปัญญา ท้อง ถัน แหล่ งา ร เรียนรู้ และ สืออน '

In [28]:
# cleaned_text_1 == cleaned_text

In [34]:
def remove_stopwords(text):
    with open('stopwords-th.txt',encoding = 'utf-8') as f:
        stopwords_list = f.read().splitlines()
    res_text = ''
    
    for token in text.split(' '):
        if token in stopwords_list:
            pass
        
        else:
            res_text += token
            res_text += ' '
    
    return res_text

In [35]:
cleaned_text_2 = remove_stopwords(cleaned_text)

In [36]:
cleaned_text_2

'ข ๕๓๓ ฆร ๑ สขฆ ๐ ได้ที มัง คัง มัน บุ คลา ร ที มีวินัย เปีย ม คุณธรรม จริยธรรม จิต สำ นึ รับผิดชอบ ตนเอง ผู้อื่น สำ นัง านศน พิจารณา ทบทวน หลัาร จุดหมาย มาตรฐาน ผลา รข เรียน รู้ที คาดหวัง เนื้อหาสาระ ทัง ลุ่ม สาระ าร เรียนรู้ หลัสู ตรา รศึษานอ ระบบ ระดับ ข ารศึษา ขัน พื่น ฐาน พุทธ ศัราช ความสอดคล้อง ตอบ สนองนโยบาย ระ ทรวง ศึษาธิารปขๆๆขมข ๓ ขๆมซ่ขมซชง ส่งผล ปรับปรุง หนังสือเรียน โด ยา ร เพิ่ม สอด แทร เนื้อหาสาระ เีย วับ อาชีพ คุณธรรม บุ ีจ ๑๓๓ จริยธรรม าร เตรียมพร้อม เข้าสู่ ประชาคม อาเซียน รายวิชา ที เียว ข้อง สัม พัน ธั์น ๑ ข ๕๑ ขสงจขขขขขขณ ๑๕ ยังคง หลัาร วิธี าร เดิม าร พัฒนา หนังสือ ที ผู้เรียน ศึษา ค้น ควำ ความรู้ ตนเอง ปฏิบัติ ๐ ข ๓ ฆ้ขขขง ๓ สิ จรรม ฝึ หัด เพือ ทดสอบ ความรู้ ความเข้าใจ าร อภิปราย แล เปลียน เรียน รู ั้บ ลุ่ม ี ๐ แข ฆ้ฆ้ศึษา เพิ่ม เต้ม จา ภูมิปัญญา ท้อง ถัน แหล่ งา ร เรียนรู้ สืออน  '

In [37]:
cleaned_text_2 == cleaned_text

False

In [38]:
#pos_tag(list_of_words)
import deepcut
cleaned_text_1 = ""
words = deepcut.tokenize(cleaned)
for word in words:
    cleaned_text_1 += word +" "


24/24 [==============================] - 1s 13ms/step


In [39]:
cleaned_text_1

'ข๕๓๓ ฆร ๑ สขฆ๐ ราย ได้ ที มังคัง และ มัน คง เป็น บุคลารที มี วินัย เปียม ไป ด้วย คุณธรรม และ จริยธรรม และ มี จิต สำนึ รับผิดชอบ ต่อ ตน เอง และ ผู้ อื่น สำนังาน ศน จึง ได้ พิจารณา ทบทวน หลัาร จุดหมาย มาตรฐาน ผลาร ขเรียนรู้ ที คาดหวัง และ เนื้อหา สาระ ทังลุ่มสาระาร เรียนรู้ ของ หลัสูตรารศึษา นอ ระบบ ระดับ ข  ารศึษา ขัน พื่น ฐาน พุทธศัราชให้ มี ความ สอดคล้อง ตอบสนอง นโยบาย ระทรวงศึษาธิาร ปข ๆ ๆ ขมข๓ ข ๆ ม ซ่ขมซ ชง ส่ง ผล ให้ ต้อง ปรับปรุง หนังสือ เรียน โดยาร เพิ่ม และ สอดแทร เนื้อหาสาระ เีย วับ อาชีพ คุณธรรม บุีจ ๑๓๓ จริยธรรม และาร เตรียม พร้อม เพื่อ เข้า สู่ ประชาคม อาเซียน ใน รายวิชา ที มี ความ เียวข้อง สัมพันธั์นแต่ ๑ ข ๕๑ ขสงจข ข ขขขขณ ๑๕ ยังคง หลัาร และ วิธีาร เดิม ในารพัฒนา หนังสือ ที ให้ ผู้ เรียน ศึษา ค้นควำ ความ รู้ ด้วย ตน เอง ปฏิบัติ ๐ ข ๓ ฆ้ ข ข ขง ๓ สิจรรม ทำ แบบ ฝึหัด เพือ ทดสอบ ความ รู้ ความ เข้าใจ มีาร อภิปราย แล เปลียน เรียนรูั้บลุ่ม หรือี๐ แขฆ้ฆ้ ศึษา เพิ่มเต้ม จา ภูมิปัญญา ท้องถัน แหล่ งาร เรียนรู้ และ สืออน '

### Pipeline

In [74]:
def preprocessing_from_folder(input_folder , output_folder):
    files = sorted(glob.glob(input_folder + '/*'))
    cleaned_corpus = ""
    for file in tqdm.tqdm(files):
        print (file)
        file_name_full  = file.split('\\')[-1]
        file_comp = file_name_full.split(' ')
        file_number, file_name = file_comp[0], file_comp[1]
#         print (file_number)
#         print (file_name)
        output_file = output_folder + '/'+ str(file_number) + ' PREPROCESSED_' + file_name
        preprocessing_pipline(file, output_file)

In [85]:
def preprocessing_pipline (file , output_file):
    # Open a file: file
    file = open(file ,mode='r',encoding = 'utf-8')
    all_of_it = file.read()
    file.close()
    
    # our built_in cleaner 
    cleaner = TextCleaner()
    cleaned = cleaner.clean_text(all_of_it)
    
    #word tokenizer
    cleaned_text = ""
    words = tokenize(cleaned)
    #words = word_tokenize(cleaned , keep_whitespace=False,engine = 'attacut')
    #words = deepcut.tokenize(cleaned)
    for word in words:
        if (len(word) == 1 or word == " "):
            continue
        else:
            cleaned_text+= word + " "
    
    #stopwords removal
    cleaned_text_nosw = remove_stopwords(cleaned_text)
    
    with open(output_file, 'w',encoding='utf-8') as f:
        f.write(f"{cleaned_text_nosw[:-1]}\n")
    

In [86]:
preprocessing_from_folder('Raw Txt Con' , 'Final Txt Con')

  0%|          | 0/3751 [00:00<?, ?it/s]

Raw Txt Con\23 schwoererd_gmail_108.txt


  0%|          | 1/3751 [00:01<1:47:55,  1.73s/it]

Raw Txt Con\24 unset0000unse_n8e1.txt


  0%|          | 2/3751 [00:11<6:37:35,  6.36s/it]

...

100%|█████████▉| 3748/3751 [1:47:11<00:04,  1.56s/it]

Raw Txt Con\997 unset0000unse_b9a6.txt
Raw Txt Con\998 unset0000unse_b9r3.txt


100%|█████████▉| 3750/3751 [1:47:11<00:00,  1.10it/s]

Raw Txt Con\999 unset0000unse_c4g5.txt


100%|██████████| 3751/3751 [1:47:12<00:00,  1.71s/it]


### Spell checking 

In [ ]:
def spell_Checking_pipline (input_folder):
    # Open a file: file
    files = sorted(glob.glob(input_folder + '/*'))
    for file in tqdm.tqdm(files):
        print (file)
        #preprocessing_pipline(file, output_file)
        f = open(file ,mode='r',encoding = 'utf-8')
        all_of_it = f.read()
        list_all_words = all_of_it.split(' ')
        f.close()
    
        spell_Correct_text = ""
        for word in list_all_words:
            if (len(word) == 1 or word == " "):
                continue
            else:
                spell_Correct_text+= correct(word) + " "    
        
        with open(file, 'w',encoding='utf-8') as f:
            f.write(f"{spell_Correct_text}\n")

### Corpus Making  

In [137]:
def make_corpus(input_path, output_file_path):
    num_to_title = get_title_dictonary()
    cleaner = TextCleaner()
    file_list = sorted(glob.glob(input_path + '/*'))
    with open(output_file_path, 'a', encoding = 'utf-8') as output_file:
        for file in tqdm.tqdm(file_list):
            with open(file, 'r', encoding = 'utf-8') as input_file:
                file_name_full  = file.split('\\')[-1]
                file_number = file_name_full.split(' ')[0]
                file_title = num_to_title[file_number]
                cleaned_title = cleaner.clean_text(file_title)
                ## in case deepcut
                file_content = ""
                #print ("befor removing spaces")
                all_file = input_file.read()
                #print (all_file)
                text = re.sub(r"\s+", " ", all_file)
                #print ("after removing spaces")
                #print (text)
                for word in text.split(" "):
                    if (len(word) == 1 or word == " "):
                        continue
                    else:
                        file_content+= word+ " "       
                file_content = file_content[:-1]
                #print ("after removing other spaces and len ==1 ")
                #print (file_content)
                file_content = file_content.replace('\n', ' ')
                # in case attcut
                #file_content = input_file.read().replace('\n', ' ')
                output_file.write(cleaned_title)
                output_file.write("$$$")
                output_file.write(file_content)
                output_file.write('\n')

In [138]:
def get_title_dictonary():
    dic = {}
    with open('num_to_title.txt',encoding = 'utf-8') as f:
        file_adresses = f.read().splitlines()
        for line in file_adresses:
            line = line.split('#')
            dic[line[0]] = str(line[1])
    return dic

In [140]:
#make_corpus ("Final Txt Con" , "Thai_corpus.txt")

In [141]:
make_corpus ("Prep Txt Con" , "Thai_corpus.txt")

100%|██████████| 5881/5881 [05:35<00:00, 17.51it/s]


example

In [133]:
file = "11 example.txt"
num_to_title = get_title_dictonary()
with open(file, 'r', encoding = 'utf-8') as input_file:
    file_name_full  = file.split('\\')[-1]
    file_number = file_name_full.split(' ')[0]
    file_title = num_to_title[file_number]
    file_content = ""
    print ("befor removing spaces")
    all_file = input_file.read()
    print (all_file)
    text = re.sub(r"\s+", " ", all_file)
    print ("after removing spaces")
    print (text)
    for word in text.split(" "):
        if (len(word) == 1 or word == " "):
            continue
        else:
            file_content+= word+ " "       
    file_content = file_content[:-1]
    print ("after removing other spaces and len ==1 ")
    print (file_content)
    file_content = file_content.replace('\n', ' ')
    #file_content = re.sub(r" +", ' ', input_file.read())
                
#                 for word in text.split(" "):
#                     if (len(word) == 1 or word == " "):
#                         continue
#                     else:
#                         file_content+= word+ " "       
#                 file_content = file_content[:-1]
#                 file_content = file_content.replace('\n', ' ')
#                 file_content = re.sub(r" +", ' ', input_file.read())
#                 # in case attcut
#                 #file_content = input_file.read().replace('\n', ' ')
#                 output_file.write(file_title)
#                 output_file.write("$$$")
#                 output_file.write(file_content)
#                 output_file.write('\n')

befor removing spaces
บรรพบุรุษ สาย สัมพันธ์ตระูลณสงขลา   ลำดับ ชั้น สาย สัมพันธ์ เครือญาติณสงขลา   พเะฅา สฺวเณคีรีสมบัติ เหยี่ยงแซ่เฮา   ภรรยา ชาว พัทลุง บุตร ๕ คน   นุจ้าพระยาอินทคีรี บุญ ทุ้ย   พระอนันตสมบัติ บุญเสี้ยว   พระยาวิเศษ ภัดีเสี้ยนจ้ง   พระนเรนทรราชาเหี้ยง   หลวงอนุรัษ์ภเบศร์แสง   เจ้าพระยาวิเชียรคีรีเม่น   โ   มั่งเรืองรุ่งมิ่งพงษ์พระยาวิชิตภัดีศรีพิชัยณรงค์ เวียง ันวงษ์เครือวัลย์  หลวงมหานุภาพ ปราบ สงคราม ผ่อง โรจนะ หัสดิน   พระศรีบุรี รัฐสิทธิ   ขุน อายุ รศาส ตรํสุนิ  สายตระูลคีรีวัฒน์รัตนปราร พระยาสุนทรานุรัษ์เนตร์  ลิ่น  ุหลาบ   ท่าน ปัน   ตงรื่น   เริง  เปรม   คล้าม   อาด จ้อยปุย ปริ่ม   อวบ   สาย จิ๋ว   ธน สาน พระยาวิเชียรคีรีชมชื่น   พระอนันตสมบัติ เอม  พระยาราชพัสดุอภิมัณฑน์ อั้น  ชุ่ม ปุย   รอด   สาย สัมพันธ์ เครือญาติณสงขลา   บันทึขุนคีรีรัฐนิคมคำขวัญ คุณวิรัชณสงขลา งาน ญาติ ๒๕๓๘ จดหมาย คุณสุชาติรัตนปราร เน้น ญาติ ใล้ ชิด สุลณสงขลา ใช้ นาม สุล สุวรรณคีรีคีรีวัตณ์รัตนปราร สงขลา รุงเทพฯ สุวรรณปัทมะจารุจินดาจารุรัตน์วัชราภัย อีหลาย สุล โรจ นะ หัส ดิน ณพัทลุง สัมพั